# Tutorial 12 - Stokes

In [ ]:
import typing

In [ ]:
import dolfinx.fem
import gmsh
import multiphenicsx.fem
import multiphenicsx.io
import multiphenicsx.mesh
import numpy as np
import petsc4py
import ufl

In [ ]:
import rbnicsx.backends

## 1. Mesh generation

In [ ]:
mesh_size = 1e-1

In [ ]:
subdomains_vertices = [
    [(1.0, 2.0), (1.0, 1.0), (2.0, 1.0)],  # subdomain 1
    [(2.0, 1.0), (2.0, 2.0), (1.0, 2.0)],  # subdomain 2
    [(0.0, 3.0), (0.0, 2.0), (1.0, 2.0)],  # subdomain 3
    [(1.0, 2.0), (1.0, 3.0), (0.0, 3.0)],  # subdomain 4
    [(0.0, 2.0), (0.0, 1.0), (1.0, 1.0)],  # subdomain 5
    [(1.0, 1.0), (1.0, 2.0), (0.0, 2.0)],  # subdomain 6
    [(0.0, 1.0), (0.0, 0.0), (1.0, 0.0)],  # subdomain 7
    [(1.0, 0.0), (1.0, 1.0), (0.0, 1.0)]  # subdomain 8
]

In [ ]:
boundaries_vertices = [
    [(2.0, 1.0), (2.0, 2.0)],  # boundary 1, inlet
    [(0.0, 0.0), (1.0, 0.0)],  # boundary 2, outlet
    [[(1.0, 2.0), (1.0, 3.0)],
     [(1.0, 3.0), (0.0, 3.0)],
     [(0.0, 3.0), (0.0, 2.0)],
     [(0.0, 2.0), (0.0, 1.0)],
     [(0.0, 1.0), (0.0, 0.0)],
     [(1.0, 0.0), (1.0, 1.0)]],  # boundary 3, fixed walls
    [[(2.0, 2.0), (1.0, 2.0)],
     [(1.0, 1.0), (2.0, 1.0)]]  # boundary 4, parametrized walls
]

In [ ]:
gmsh.initialize()
gmsh.model.add("stokes")

In [ ]:
gmsh_points = dict()
for subdomain_vertices in subdomains_vertices:
    for vertex in subdomain_vertices:
        try:
            gmsh_points[vertex]
        except KeyError:
            gmsh_points[vertex] = gmsh.model.geo.addPoint(vertex[0], vertex[1], 0.0, mesh_size)

In [ ]:
gmsh_lines = dict()
gmsh_subdomains = list()
for subdomain_vertices in subdomains_vertices:
    subdomain_lines = list()
    for v in range(3):
        key = (gmsh_points[subdomain_vertices[v]], gmsh_points[subdomain_vertices[(v + 1) % 3]])
        try:
            gmsh_lines[key]
        except KeyError:
            gmsh_lines[key] = gmsh.model.geo.addLine(*key)
            gmsh_lines[key[1], key[0]] = - gmsh_lines[key]
        subdomain_lines.append(gmsh_lines[key])
    subdomain_lines_loop = gmsh.model.geo.addCurveLoop(subdomain_lines)
    gmsh_subdomains.append(gmsh.model.geo.addPlaneSurface([subdomain_lines_loop]))

In [ ]:
gmsh.model.geo.synchronize()
for (label, gmsh_subdomain) in enumerate(gmsh_subdomains):
    gmsh.model.addPhysicalGroup(2, [gmsh_subdomain], label + 1)
for (label, boundary) in enumerate(boundaries_vertices):
    if isinstance(boundary[0], tuple):
        gmsh.model.addPhysicalGroup(
            1, [gmsh_lines[gmsh_points[boundary[0]], gmsh_points[boundary[1]]]], label + 1)
    elif isinstance(boundary[0], list):
        gmsh.model.addPhysicalGroup(
            1, [gmsh_lines[gmsh_points[boundary_[0]], gmsh_points[boundary_[1]]] for boundary_ in boundary],
            label + 1)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

## 2. Problem definition

In [ ]:
class AffineShapeParametrization(rbnicsx.backends.MeshMotion):
    """Deform the domain with an affine shape parametrization."""

    def __init__(self, mu: np.typing.NDArray[np.float64]) -> None:
        # Define function space
        M = dolfinx.fem.VectorFunctionSpace(mesh, ("Lagrange", mesh.geometry.cmap.degree))
        # Interpolate affine shape parametrization expression on a dolfinx Function
        shape_parametrization = dolfinx.fem.Function(M)
        shape_parametrization.interpolate(
            lambda x: (
                mu[4] * (x[0] - 1) + mu[1], mu[0] * (x[1] - 1) + mu[4] * np.tan(mu[5]) * (x[0] - 1) + mu[2]),
            subdomains.indices[np.isin(subdomains.values, (1, 2))])
        shape_parametrization.interpolate(
            lambda x: (mu[1] * x[0], mu[3] * (x[1] - 2) + mu[2] + mu[0]),
            subdomains.indices[np.isin(subdomains.values, (3, 4))])
        shape_parametrization.interpolate(
            lambda x: (mu[1] * x[0], mu[0] * (x[1] - 1) + mu[2]),
            subdomains.indices[np.isin(subdomains.values, (5, 6))])
        shape_parametrization.interpolate(
            lambda x: (mu[1] * x[0], mu[2] * x[1]),
            subdomains.indices[np.isin(subdomains.values, (7, 8))])
        # Initialize mesh motion object
        super().__init__(mesh, shape_parametrization)

In [ ]:
mu_mesh_motion = np.array([0.5, 1.5, 0.75, 1.5, 1.25, np.pi / 6])
with AffineShapeParametrization(mu_mesh_motion):
    multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
class Problem(object):
    """Define a linear problem, and solve it with KSP."""

    def __init__(self) -> None:
        # Define function space
        V_element = ufl.VectorElement("Lagrange", mesh.ufl_cell(), 2)
        Q_element = ufl.FiniteElement("Lagrange", mesh.ufl_cell(), 1)
        V = dolfinx.fem.FunctionSpace(mesh, V_element)
        Q = dolfinx.fem.FunctionSpace(mesh, Q_element)
        self._VQ = (V, Q)
        # Define trial and test functions
        (v, q) = (ufl.TestFunction(V), ufl.TestFunction(Q))
        (u, p) = (ufl.TrialFunction(V), ufl.TrialFunction(Q))
        # Define bilinear form of the problem
        lhs = [
            [ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx, - ufl.inner(p, ufl.div(v)) * ufl.dx],
            [ufl.inner(ufl.div(u), q) * ufl.dx, None]
        ]
        self._lhs = lhs
        self._lhs_cpp = dolfinx.fem.form(lhs)
        # Define linear form of the problem
        gravity = dolfinx.fem.Constant(mesh, np.array([0.0, -10.0], petsc4py.PETSc.ScalarType))
        zero_scalar = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0))
        rhs = [ufl.inner(gravity, v) * ufl.dx, ufl.inner(zero_scalar, q) * ufl.dx]
        self._rhs = rhs
        self._rhs_cpp = dolfinx.fem.form(rhs)
        # Define boundary conditions for the problem
        zero_vector = dolfinx.fem.Constant(mesh, np.zeros(mesh.topology.dim, petsc4py.PETSc.ScalarType))
        facets_walls = boundaries.indices[np.isin(boundaries.values, (3, 4))]
        bdofs_V_walls = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, facets_walls)
        bcs = [dolfinx.fem.dirichletbc(zero_vector, bdofs_V_walls, V)]
        self._bcs = bcs
        # Prepare storage for mesh motion object
        self._mesh_motion = None

    @property
    def function_spaces(self) -> typing.Tuple[dolfinx.fem.FunctionSpace, dolfinx.fem.FunctionSpace]:
        """Return the function spaces of the problem."""
        return self._VQ

    @property
    def bilinear_block_form(self) -> typing.List[typing.List[ufl.Form]]:
        """Return the bilinear block form of the problem."""
        return self._lhs

    @property
    def linear_block_form(self) -> typing.List[ufl.Form]:
        """Return the linear block form of the problem."""
        return self._rhs

    @property
    def boundary_conditions(self) -> typing.List[dolfinx.fem.DirichletBCMetaClass]:
        """Return the boundary conditions for the problem."""
        return self._bcs

    @property
    def mesh_motion(self) -> rbnicsx.backends.MeshMotion:
        """Return the mesh motion object that was used in the latest solve."""
        return self._mesh_motion

    def _assemble_matrix(self) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side matrix."""
        A = dolfinx.fem.assemble_matrix_block(self._lhs_cpp, bcs=self._bcs)
        A.assemble()
        return A

    def _assemble_vector(self) -> petsc4py.PETSc.Vec:
        """Assemble the right-hand side vector."""
        return dolfinx.fem.assemble_vector_block(self._rhs_cpp, self._lhs_cpp, bcs=self._bcs)

    def solve(self, mu: np.typing.NDArray[np.float64]) -> dolfinx.fem.Function:
        """Apply shape parametrization and solve the problem."""
        with AffineShapeParametrization(mu) as self._mesh_motion:
            return self._solve()

    def _solve(self) -> dolfinx.fem.Function:
        """Solve the linear problem with KSP."""
        A = self._assemble_matrix()
        F = self._assemble_vector()
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(A)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.getPC().setFactorSolverType("mumps")
        ksp.setFromOptions()
        solution = dolfinx.fem.create_vector_block(self._rhs_cpp)
        ksp.solve(F, solution)
        solution.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        (solution_u, solution_p) = (dolfinx.fem.Function(self._VQ[0]), dolfinx.fem.Function(self._VQ[1]))
        with multiphenicsx.fem.BlockVecSubVectorWrapper(
            solution, [c.function_space.dofmap for c in (solution_u, solution_p)]
        ) as solution_wrapper:
            for solution_wrapper_local, component in zip(solution_wrapper, (solution_u, solution_p)):
                with component.vector.localForm() as component_local:
                    component_local[:] = solution_wrapper_local
        return (solution_u, solution_p)

In [ ]:
problem = Problem()

In [ ]:
mu_solve = np.array([0.5, 1.5, 0.75, 1.5, 1.25, np.pi / 6])
(solution_u, solution_p) = problem.solve(mu_solve)

In [ ]:
with problem.mesh_motion:
    multiphenicsx.io.plot_vector_field(solution_u, "high fidelity velocity", glyph_factor=1)

In [ ]:
with problem.mesh_motion:
    multiphenicsx.io.plot_scalar_field(solution_p, "high fidelity pressure")